In [ ]:
# Descriptive statistics and figures based on the wrangled dataset from tre-test-prep
# Input data : participant-data.pkl / participant-data.csv etc. created in script 1-tre-test-prep 

In [ ]:
# import libraries 

# please try to to import libraries stored in another jupyter notebook
# this notebook is called 'my_libraries'
# and is imported as follows

import import_ipynb
from my_libraries import *

In [ ]:
# if importing via a notebook does not work, please uncomment the below

#import io, os, sys, types
#from IPython import get_ipython
#from nbformat import read
#from IPython.core.interactiveshell import InteractiveShell
#import numpy as np
#import pandas as pd
#from datetime import date
#from scipy import stats
#from scipy.stats import norm
#import seaborn as sns
#import matplotlib
#import matplotlib.pyplot as plt
#import pickle
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import StratifiedShuffleSplit
#from sklearn.metrics import confusion_matrix, accuracy_score
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.model_selection import train_test_split
#from sklearn import metrics
#import sklearn.metrics as metrics
#from sklearn.metrics import classification_report
#sns.set_palette("BuPu")


In [ ]:
# set directories 

os.chdir('/Users/bethanyisaacs/Documents/git-new/tre-demo-epi-test/DemoEpidemiologyScenario/')

indir = os.getcwd()+'/OFH-data'
outdir = os.getcwd()+'/_outputs'

# read in from pickle

f = open(outdir+"/participant-data.pkl",'rb') 
dat = pickle.load(f)[0]

In [ ]:
# get value counts for diabetic participants per data source where 1 == diabetes 

d_counts = dat.groupby(['diabetic status', 'diabetic source'])['diabetic source'].size().to_frame()

# save as a table 

d_counts.to_csv(outdir+'/diabetic_counts.csv')

d_counts

In [ ]:
# plot 1 : density histrogram of gender by year of birth  

sns.set_palette("magma")

f, axs = plt.subplots(1,4,figsize=(25,5),sharey=False, sharex=False) 

# density plot for sex x year of birth for diabetic participants 

p1 = dat[dat['diabetic status'] == 1].groupby('sex')['year of birth'].plot(kind='kde', title = "sex x year of birth : diabetic participants", ax=axs[0], legend=True)

# density plot for sex x year of birth for non diabetic participants 

p2 = dat[dat['diabetic status'] == 0].groupby('sex')['year of birth'].plot(kind='kde', title = "sex x year of birth : non diabetic participants", ax=axs[2]) 

# density plot with kde and normal distribution for year of birth with normal distribution for diabetic participants

p3 = sns.histplot(dat[dat['diabetic status'] == 1]['year of birth'].astype(int), bins=30, stat='density', color='orange', ax=axs[1])
sns.kdeplot(dat[dat['diabetic status'] == 1]['year of birth'], color='purple', ax=axs[1]).set( title = "year of birth x norm dist : diabetic participants")
mu, std = stats.norm.fit(dat[dat['diabetic status'] == 1]['year of birth'])
p3.plot(np.linspace(*p3.get_xlim(), 100), stats.norm.pdf(np.linspace(*p3.get_xlim(), 100), mu.astype(int), std.astype(int)), color='blue')

# density plot with kde and normal distribution for year of birth with normal distribution for diabetic participants

p4 = sns.histplot(dat[dat['diabetic status'] == 0]['year of birth'].astype(int), bins=30, stat='density', color='orange', ax=axs[3])
sns.kdeplot(dat[dat['diabetic status'] == 0]['year of birth'], color='purple', ax=axs[3]).set( title = "year of birth x norm dist : non diabetic participants")
mu, std = stats.norm.fit(dat[dat['diabetic status'] == 0]['year of birth'])
p4.plot(np.linspace(*p4.get_xlim(), 100), stats.norm.pdf(np.linspace(*p3.get_xlim(), 100), mu.astype(int), std.astype(int)), color='blue')

# save plot as png 

plt.savefig(outdir+'/histo-plots.png', facecolor='white', bbox_inches="tight")


In [ ]:
# create a descriptive statistics table

num_dat_desc = dat.select_dtypes(include=np.number).drop(columns='diabetic status').describe().round(0) # describe numeric columns and store as dataframe

cat_dat_desc = dat.loc[:, dat.dtypes == object].apply(lambda x: x.value_counts()).T.stack().to_frame().reset_index().rename(columns={'level_0' : 'question', 'level_1': 'answer', 0:'counts'}) # describe categorical data (as total counts per level) and store as dataframe

cat_dat_desc_dic = dict(tuple(cat_dat_desc.groupby('question'))) # split categorical data in to dictionaries of dataframes per question : due to long format // queryable by 'question' - which is actually source. e.g. cat_dat_desc_dic['sex']

In [ ]:
num_dat_desc

In [ ]:
cat_dat_desc

In [ ]:
cat_dat_desc_dic['sex']

In [ ]:
# # # save descriptive statistics tables 

num_dat_desc.to_csv(outdir+'/desc_stats_num.csv')
cat_dat_desc.to_csv(outdir+'/desc_stats_cat.csv')

with open(outdir+'/desc_stats_cat_dic.pkl', 'wb') as pickle_file:
    pickle.dump([cat_dat_desc], pickle_file)